In [ ]:
'''
1. resample the test results by bootstrapping
'''
from __future__ import absolute_import
from __future__ import print_function

from models.metrics import print_metrics_binary

import sklearn.utils as sk_utils
import numpy as np
import pandas as pd
import argparse
import json
import os

import sys; sys.argv=['']; del sys

parser = argparse.ArgumentParser()
parser.add_argument('--save_file', type=str, default='results.json')
args = parser.parse_args()

args.n_iters=10000 # number of bootstrapped samples from the test data

# illustrative prediction data 
args.prediction="test_predictions/k_gru.n16.d0.3.dep2.bs8.ts1.0.epoch10.test0.2858196917729375.state.csv"

pred_df = pd.read_csv(args.prediction, index_col=False)
args.test_listfile='s3://aws-glue-scripts-271538242229-us-west-2/data/in-hospital-mortality/test/listfile.csv' # this includes targets

test_df = pd.read_csv(args.test_listfile, index_col=False)

df = test_df.merge(pred_df, left_on='stay', right_on='stay', how='left', suffixes=['_l', '_r'])
assert (df['prediction'].isnull().sum() == 0)
assert (df['y_true_l'].equals(df['y_true_r']))

metrics = [('AUC of ROC', 'auroc'),
           ('AUC of PRC', 'auprc'),
           ('min(+P, Se)', 'minpse')]

data = np.zeros((df.shape[0], 2))
data[:, 0] = np.array(df['prediction'])
data[:, 1] = np.array(df['y_true_l'])

results = dict()
results['n_iters'] = args.n_iters
ret = print_metrics_binary(data[:, 1], data[:, 0], verbose=0)
for (m, k) in metrics:
    results[m] = dict()
    results[m]['value'] = ret[k]
    results[m]['runs'] = []

for i in range(args.n_iters):
    cur_data = sk_utils.resample(data, n_samples=len(data))
    ret = print_metrics_binary(cur_data[:, 1], cur_data[:, 0], verbose=0)
    for (m, k) in metrics:
        results[m]['runs'].append(ret[k])

for (m, k) in metrics:
    runs = results[m]['runs']
    results[m]['mean'] = np.mean(runs)
    results[m]['median'] = np.median(runs)
    results[m]['std'] = np.std(runs)
    results[m]['2.5% percentile'] = np.percentile(runs, 2.5)
    results[m]['97.5% percentile'] = np.percentile(runs, 97.5)
    results[m]['25% percentile'] = np.percentile(runs, 25)
    results[m]['75% percentile'] = np.percentile(runs, 75)
    
print("Saving the results in {} ...".format(args.save_file))
with open(args.save_file, 'w') as f:
    json.dump(results, f)



In [ ]:

# read resampled data

file6='ihm_results.json'
# file='ihm_results_lstm.json'
# file2='ihm_results_lr.json'
# file3='ihm_results_lr_24.json'
# file4='ihm_results_lstm_bi_tr_16.json'
# file5='ihm_results_lstm_channelwise_dep_16.json'
# file7='ihm_results_gru_tr.json'
# file8='ihm_results_gru_channelwise_tr.json'
# file9='ihm_results_gru_channelwise.json'


# summary_lstm = pd.read_json(file) 
# summary_lr = pd.read_json(file2)
# summary_lr24 = pd.read_json(file3)

# summary_lstm_tr = pd.read_json(file4)
# summary_lstm_cw = pd.read_json(file5)
# summary_gru_tr = pd.read_json(file7)

summary_gru = pd.read_json(file6)


In [ ]:

import itertools
from matplotlib.cbook import _reshape_2D
import matplotlib.pyplot as plt
import numpy as np

def my_boxplot_stats(X, whis=1.5, bootstrap=None, labels=None,
                  autorange=False, percents=[25, 75]):

    def _bootstrap_median(data, N=5000):
        # determine 95% confidence intervals of the median
        M = len(data)
        percentiles = [2.5, 97.5]

        bs_index = np.random.randint(M, size=(N, M))
        bsData = data[bs_index]
        estimate = np.median(bsData, axis=1, overwrite_input=True)

        CI = np.percentile(estimate, percentiles)
        return CI

    def _compute_conf_interval(data, med, iqr, bootstrap):
        if bootstrap is not None:
            # Do a bootstrap estimate of notch locations.
            # get conf. intervals around median
            CI = _bootstrap_median(data, N=bootstrap)
            notch_min = CI[0]
            notch_max = CI[1]
        else:

            N = len(data)
            notch_min = med - 1.57 * iqr / np.sqrt(N)
            notch_max = med + 1.57 * iqr / np.sqrt(N)

        return notch_min, notch_max

    # output is a list of dicts
    bxpstats = []

    # convert X to a list of lists
    X = _reshape_2D(X, "X")

    ncols = len(X)
    if labels is None:
        labels = itertools.repeat(None)
    #elif len(labels) != ncols:
    #    raise ValueError("Dimensions of labels and X must be compatible")

    input_whis = whis
    #for ii, (x, label) in enumerate(zip(X, labels)):
    
    # empty dict
    stats = {}
    if labels is not None:
        stats['label'] = labels

    # restore whis to the input values in case it got changed in the loop
    whis = input_whis

    # note tricksyness, append up here and then mutate below
    bxpstats.append(stats)

    # if empty, bail
    #if len(x) == 0:
    stats['fliers'] = np.array([])
    stats['mean'] = np.nan
    stats['med'] = np.nan
    stats['q1'] = np.nan
    stats['q3'] = np.nan
    stats['cilo'] = np.nan
    stats['cihi'] = np.nan
    stats['whislo'] = np.nan
    stats['whishi'] = np.nan
    stats['med'] = np.nan

    x = np.asarray(X)[0]
    stats['mean'] = np.mean(x)

    # median
    med = np.percentile(x, 50)
    q1, q3 = np.percentile(x, (percents[0], percents[1]))

    stats['iqr'] = q3 - q1
    if stats['iqr'] == 0 and autorange:
        whis = 'range'
    stats['cilo'], stats['cihi'] = _compute_conf_interval(
        x, med, stats['iqr'], bootstrap
    )

    if np.isscalar(whis):
        if np.isreal(whis):
            loval = q1 - whis * stats['iqr']
            hival = q3 + whis * stats['iqr']
        elif whis in ['range', 'limit', 'limits', 'min/max']:
            loval = np.min(x)
            hival = np.max(x)
        else:
            raise ValueError('whis must be a float, valid string, or list '
                             'of percentiles')
    else:
        loval = np.percentile(x, whis[0])
        hival = np.percentile(x, whis[1])

    wiskhi = np.compress(x <= hival, x)
    if len(wiskhi) == 0 or np.max(wiskhi) < q3:
        stats['whishi'] = q3
    else:
        stats['whishi'] = np.max(wiskhi)

    wisklo = np.compress(x >= loval, x)
    if len(wisklo) == 0 or np.min(wisklo) > q1:
        stats['whislo'] = q1
    else:
        stats['whislo'] = np.min(wisklo)

    stats['fliers'] = np.hstack([
        np.compress(x < stats['whislo'], x),
        np.compress(x > stats['whishi'], x)
    ])

    stats['q1'], stats['med'], stats['q3'] = q1, med, q3

    return bxpstats

stats = {}

# Compute the boxplot stats with our desired percentiles
stats['G'] = my_boxplot_stats(summary_gru.loc["runs"][1], labels='gru', percents=[2.5, 97.5])[0]
# stats['A'] = my_boxplot_stats(summary_lr.loc["runs"][1], labels='lr', percents=[2.5, 97.5])[0]
# stats['B'] = my_boxplot_stats(summary_lr24.loc["runs"][1], labels='lr24', percents=[2.5, 97.5])[0]
# stats['C'] = my_boxplot_stats(summary_lstm.loc["runs"][1], labels='lstm', percents=[2.5, 97.5])[0]
# stats['D'] = my_boxplot_stats(summary_lstm_tr.loc["runs"][1], labels='lstm_tr', percents=[2.5, 97.5])[0]
# stats['E'] = my_boxplot_stats(summary_lstm_cw.loc["runs"][1], labels='lstm_cw', percents=[2.5, 97.5])[0]
# stats['H'] = my_boxplot_stats(summary_gru_tr.loc["runs"][1], labels='gru_tr', percents=[2.5, 97.5])[0]
# stats['I'] = my_boxplot_stats(summary_gru_cw["runs"][1], labels='gru_cw', percents=[2.5, 97.5])[0]
# stats['J'] = my_boxplot_stats(summary_gru_cw_tr["runs"][1], labels='gru_cw_tr', percents=[2.5, 97.5])[0]
# stats['F'] = my_boxplot_stats(summary_lstm_cw_tr["runs"][1], labels='lstm_cw_tr', percents=[2.5, 97.5])[0]

fig, ax = plt.subplots(1, 1)
# Plot boxplots from computed statistics
bp = ax.bxp([stats['G']], positions=range(1), vert=False)
#bp = ax.bxp([stats['A'], stats['B'], stats['C'], stats['D'], stats['E'], stats['F'],stats['G'],stats['H'],stats['I'],stats['J']], positions=range(10), vert=False)

# Colour the lines in the boxplot blue
for element in bp.keys():
    plt.setp(bp[element], color='C0')

In [ ]:
'''
2. visualizing calibration (predicted vs observed mortality)
'''
try:
    # lr 
    d=pd.read_csv("models/logistic/predictions/all.all.l2.C0.001_48.csv")

    #channel wise lstm
    d2=pd.read_csv("models/test_predictions/k_channel_wise_lstms.n16.szc4.0.d0.3.dep2.bs8.ts1.0.epoch10.test0.2853237935876639.state.csv")

    # lstm with tr
    d3=pd.read_csv("models/test_predictions/k_lstm.n16.d0.3.dep2.bs8.ts1.0.trc0.5.epoch10.test0.2944733425188627.state.csv")

    # gru with tr 
    d5=pd.read_csv("models/test_predictions/k_gru.n16.d0.3.dep2.bs8.ts1.0.trc0.5.epoch10.test0.28975883245579137.state.csv")

    # lstm
    d6=pd.read_csv("test_predictions/k_lstm.n16.d0.3.dep2.bs8.ts1.0.epoch10.test0.294850016698417.state.csv")

except:
    # gru 
    pass
d4=pd.read_csv("test_predictions/k_gru.n16.d0.3.dep2.bs8.ts1.0.epoch10.test0.2858196917729375.state.csv")

try: 
    d=d.sort_values('prediction')
    d2=d2.sort_values('prediction')
    d3=d3.sort_values('prediction')
    d5=d5.sort_values('prediction')
    d6=d6.sort_values('prediction')
except:
    pass
d4=d4.sort_values('prediction')
    
import numpy as np
bins = [x for x in np.linspace(0,1,11)]

try:

    d['binned'] = pd.cut(d['prediction'], bins)
    d2['binned'] = pd.cut(d2['prediction'], bins)
    d3['binned'] = pd.cut(d3['prediction'], bins)
    d5['binned'] = pd.cut(d5['prediction'], bins)
    d6['binned'] = pd.cut(d6['prediction'], bins)
except:
    pass

d4['binned'] = pd.cut(d4['prediction'], bins)

try:
    g =(d.groupby('binned')
          .agg({'y_true':['mean'],'prediction':['mean'] }))

    g2 =(d2.groupby('binned')
          .agg({'y_true':['mean'],'prediction':['mean'] }))

    g3 =(d3.groupby('binned')
          .agg({'y_true':['mean'],'prediction':['mean'] }))

    g5 =(d5.groupby('binned')
          .agg({'y_true':['mean'],'prediction':['mean'] }))

    g6 =(d6.groupby('binned')
          .agg({'y_true':['mean'],'prediction':['mean'] }))
    
except:
    pass
g4 =(d4.groupby('binned')
      .agg({'y_true':['mean'],'prediction':['mean'] }))




In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(6, 6))
try:
    ax.scatter(g['prediction'], g['y_true'], c=".3", label="logistic")
    ax.scatter(g2['prediction'], g2['y_true'], c="red", label="channel-wise lstm")

    ax.scatter(g3['prediction'], g3['y_true'], c="orange", label="lstm with tr")

    ax.scatter(g5['prediction'], g5['y_true'], c="purple", label="gru with tr")
    ax.scatter(g6['prediction'], g6['y_true'], c="blue", label="lstm")
except:
    pass
ax.scatter(g4['prediction'], g4['y_true'], c="cyan", label="gru")

ax.plot([0, 1], [0, 1], ls="--", c=".3")
ax.set_xlabel('predicted probability')     
ax.set_ylabel('observed probability')     

ax.legend()

In [ ]:
'''
3. ROC curves
'''
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

try:
    #lr
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = roc_curve(d.iloc[:,2], d.iloc[:,1])
    roc_auc = auc(fpr, tpr)

    #cw lstm
    roc_auc2 = dict()
    fpr2, tpr2, _ = roc_curve(d2.iloc[:,2], d2.iloc[:,1])
    roc_auc2 = auc(fpr2, tpr2)

    #lstm tr
    roc_auc3 = dict()
    fpr3, tpr3, _ = roc_curve(d3.iloc[:,2], d3.iloc[:,1])
    roc_auc3 = auc(fpr3, tpr3)

    #gru tr

    roc_auc5 = dict()
    fpr5, tpr5, _ = roc_curve(d5.iloc[:,2], d5.iloc[:,1])
    roc_auc5 = auc(fpr5, tpr5)

    #lstm

    roc_auc6 = dict()
    fpr6, tpr6, _ = roc_curve(d6.iloc[:,2], d6.iloc[:,1])
    roc_auc6 = auc(fpr6, tpr6)
except:
    pass
#gru
roc_auc4 = dict()
fpr4, tpr4, _ = roc_curve(d4.iloc[:,2], d4.iloc[:,1])
roc_auc4 = auc(fpr4, tpr4)

import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(6, 6))
try:
    ax.plot(fpr, tpr, color='black',
             lw=2, label='ROC curve (area = %0.3f), LR' % roc_auc)
    ax.plot(fpr2, tpr2, color='red',
             lw=2, label='ROC curve (area = %0.3f), channel wise lstm' % roc_auc2)
    ax.plot(fpr3, tpr3, color='green',
             lw=2, label='ROC curve (area = %0.3f), lstm tr' % roc_auc3)
    ax.plot(fpr5, tpr5, color='purple',
             lw=2, label='ROC curve (area = %0.3f), gru tr' % roc_auc5)
    ax.plot(fpr6, tpr6, color='cyan',
             lw=2, label='ROC curve (area = %0.3f), lstm' % roc_auc6)
except:
    pass
ax.plot(fpr4, tpr4, color='orange',
         lw=2, label='ROC curve (area = %0.3f), gru' % roc_auc4)

ax.plot([0, 1], [0, 1], ls="--", c=".3")

ax.set_xlabel('fpr')     
ax.set_ylabel('tpr')     
ax.legend(loc="upper left")

ax.legend()


In [ ]:
'''
4. PR curves
'''
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

def my_fun(x):
    if x['prediction']>0.5 :
        x['yhat']=1
    else:
        x['yhat']=0
    return x  
 
try:
    d2=d2.apply(lambda x: my_fun(x), axis=1)
    d3=d3.apply(lambda x: my_fun(x), axis=1)
    d5=d5.apply(lambda x: my_fun(x), axis=1)
    d6=d6.apply(lambda x: my_fun(x), axis=1)
    d2=d2.apply(lambda x: my_fun(x), axis=1)
    d=d.apply(lambda x: my_fun(x), axis=1)
except:
    pass
d4=d4.apply(lambda x: my_fun(x), axis=1)

try:
    lr_precision, lr_recall, _ = precision_recall_curve(d.iloc[:,2], d.iloc[:,1])
    lr_f1, lr_auc = f1_score(d['yhat'], d['y_true']), auc(lr_recall, lr_precision)

    lr_precision2, lr_recall2, _ = precision_recall_curve(d2.iloc[:,2], d2.iloc[:,1])
    lr_f12, lr_auc2 = f1_score(d2['yhat'], d2['y_true']), auc(lr_recall2, lr_precision2)

    lr_precision3, lr_recall3, _ = precision_recall_curve(d3.iloc[:,2], d3.iloc[:,1])
    lr_f13, lr_auc3 = f1_score(d3['yhat'], d3['y_true']), auc(lr_recall3, lr_precision3)

    lr_precision5, lr_recall5, _ = precision_recall_curve(d5.iloc[:,2], d5.iloc[:,1])
    lr_f15, lr_auc5 = f1_score(d5['yhat'], d5['y_true']), auc(lr_recall5, lr_precision5)

    lr_precision6, lr_recall6, _ = precision_recall_curve(d6.iloc[:,2], d6.iloc[:,1])
    lr_f16, lr_auc6 = f1_score(d6['yhat'], d6['y_true']), auc(lr_recall6, lr_precision6)
except:
    pass
lr_precision4, lr_recall4, _ = precision_recall_curve(d4.iloc[:,2], d4.iloc[:,1])
lr_f14, lr_auc4 = f1_score(d4['yhat'], d4['y_true']), auc(lr_recall4, lr_precision4)

f, ax = plt.subplots(figsize=(6, 6))
try:
    ax.plot(lr_recall, lr_precision, color='black', lw=2, label='AUC PR (area = %0.3f), LR' % lr_auc)
    ax.plot(lr_recall2, lr_precision2, color='red', lw=2, label='AUC PR (area = %0.3f), channel wise lstm' % lr_auc2)
    ax.plot(lr_recall3, lr_precision3, color='green', lw=2, label='AUC PR (area = %0.3f), lstm tr' % lr_auc3)
    ax.plot(lr_recall5, lr_precision5, color='purple', lw=2, label='AUC PR (area = %0.3f), gru tr' % lr_auc5)
    ax.plot(lr_recall6, lr_precision6, color='cyan', lw=2, label='AUC PR (area = %0.3f), lstm' % lr_auc6)
except:
    pass
ax.plot(lr_recall4, lr_precision4, color='orange', lw=2, label='AUC PR (area = %0.3f), gru' % lr_auc4)

# axis labels
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

ax.plot([1, 1], [0, 0], ls="--", c=".3")

# show the legend
ax.legend()
 